In [1]:
#import requests
import polars as pl
from fyers_apiv3 import fyersModel
import json
import datetime as dt
from dateutil.relativedelta import relativedelta
import pyotp
#from selenium import webdriver
#from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.chrome.options import Options
import time
import pickle
import pyperclip
import webbrowser
from urllib.parse import parse_qs,urlparse
import os

print(os.getlogin())

fd=pl.read_parquet(f'C:\\Users\\{os.getlogin()}\\OneDrive\\Python\\stockdata.parquet')
with open(f"C:\\Users\\{os.getlogin()}\\Downloads\\fyers.json") as f:
    f=json.load(f)  
# Only once for an app
#session = fyersModel.SessionModel(client_id = f['client_id'],secret_key = f['secret_key'],redirect_uri = f['redirect_uri'],response_type = f['response_type'],state = f['state'] )
#a_c = session.generate_authcode()


mswas


In [2]:
def generate_access_token() :
    global user_id, pin, app_id, api_key, redirect_url, totp_key, url
    print("\nGenerating Access Token .....................") 
    try :
        user_id = f['ci']
        pin = f['pin']
        app_id = f['client_id']
        api_key = f['secret_key']
        redirect_url = f['redirect_uri']
        totp_key = f['totp']
        session = fyersModel.SessionModel(
        client_id = app_id,
        secret_key = api_key,
        redirect_uri = redirect_url,
        response_type = "code",
        state=f['state'])
        url = session.generate_authcode()
        webbrowser.open_new(url)
        old_clipboard_contents = pyperclip.paste()
        new_clipboard_contents = pyperclip.paste()
        while old_clipboard_contents == new_clipboard_contents : 
            new_clipboard_contents = pyperclip.paste()
            time.sleep(1)
        url = new_clipboard_contents
        parsed = urlparse(url)
        auth_code = parse_qs(parsed.query)["auth_code"][0]
        session = fyersModel.SessionModel(client_id = app_id,secret_key = api_key,redirect_uri = redirect_url,response_type = "code",grant_type = "authorization_code")
        session.set_token(auth_code)
        response = session.generate_token()
        access_token = response["access_token"]
        refresh_token = response["refresh_token"]
        token_dict = {"app_id": app_id,"access_token": access_token,"refresh_token": refresh_token}
        with open ("token_dict.pickle", "wb") as file :
            pickle.dump(token_dict, file)
    except :
        time.sleep(2)
        generate_access_token()
def fyers_login() :
    global fyers, token_dict, name
    while True:
        try:
            with open("token_dict.pickle", "rb") as file :
                token_dict = pickle.load(file)
                #print(token_dict)
        except :
            token_dict = {"app_id": 0, "access_token": 0, "refresh_token": 0}
        fyers = fyersModel.FyersModel(client_id=token_dict["app_id"], is_async=False, token= token_dict["access_token"], log_path='')
        response = fyers.get_profile()
        if response["s"] == "error" :
            generate_access_token()
        else :
            print("\nlogin Details ..........")
            print(response["data"])
            name = response["data"]["name"]
            #print(name)
            print("")
            print()
            break
fyers_login()


Generating Access Token .....................

login Details ..........
{'fy_id': 'YS42083', 'name': 'SWASTIK MISHRA', 'image': None, 'display_name': None, 'pin_change_date': '18-09-2024 15:17:23', 'email_id': 'm.swastik@gmail.com', 'pwd_change_date': None, 'PAN': '---------', 'mobile_number': '7506230285', 'totp': True, 'pwd_to_expire': 90}




In [23]:
ny=5
sym="TCS"
try:
    print('try')
    sd=fd.filter(pl.col('symbol')==sym)['epoch'].max().strftime('%Y-%m-%d')
    ed=(dt.datetime.now()-dt.timedelta(hours=24)).strftime('%Y-%m-%d')
    std=fyers.history(data={'symbol':f'NSE:{sym}-EQ','resolution':'1D','date_format':1,'range_from':sd,'range_to':ed,'cont_flag':1})
    df=pl.DataFrame(std['candles'],schema={'epoch':pl.Int32,'open':pl.Float32,'high':pl.Float32,'low':pl.Float32,'close':pl.Float32,'volume':pl.Float32},orient="row")
    df=df.with_columns(epoch=pl.from_epoch('epoch'))
    df=df.with_columns(symbol=pl.lit(sym))
    fd=pl.concat([fd,df])
    fd.write_parquet('C:\\Users\\smishra14\\OneDrive\\Python\\stockdata.parquet')
except:
    print('except')
    for i in range(ny):
        print('loop')
        ed=(dt.datetime.now()-dt.timedelta(hours=24)-relativedelta(years=i)).strftime('%Y-%m-%d')
        sd=(dt.datetime.now()-dt.timedelta(hours=24)-relativedelta(years=i+1)).strftime('%Y-%m-%d')
        std=fyers.history(data={'symbol':f'NSE:{sym}-EQ','resolution':'1D','date_format':1,'range_from':sd,'range_to':ed,'cont_flag':1})
        df=pl.DataFrame(std['candles'],schema={'epoch':pl.Int32,'open':pl.Float32,'high':pl.Float32,'low':pl.Float32,'close':pl.Float32,'volume':pl.Float32},orient="row")
        df=df.with_columns(epoch=pl.from_epoch('epoch'))
        df=df.with_columns(symbol=pl.lit(sym))
        fd=pl.concat([fd,df])
    fd.write_parquet('C:\\Users\\smishra14\\OneDrive\\Python\\stockdata.parquet')

try
except
loop
loop
loop
loop
loop


In [3]:
import json
import pyotp
with open("C:\\Users\\mswas\\Downloads\\fyers.json") as f:
    f=json.load(f)
pyotp.TOTP(f['totp']).now()

'879106'

In [2]:
from pyecharts.charts import Bar, Candlestick, Kline
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

sym="TCS"
df=fd.filter(pl.col('symbol')==sym).sort('epoch',descending=False)
df=df.with_columns(gain=pl.when(pl.col('open')<pl.col('close')).then(1).otherwise(-1))
bar = (
    Candlestick(init_opts=opts.InitOpts(height="600px",width="1320px"))
    .add_xaxis(df['epoch'].cast(pl.String).to_list())
    .add_yaxis(series_name="", y_axis=df['open','close','low','high'].to_numpy().tolist(),)
    #.add_dataset(data=list(df['open','high','low','close','gain'].to_numpy().tolist()),)
    .set_global_opts(title_opts=opts.TitleOpts(title=f"{sym}"), datazoom_opts=[
                opts.DataZoomOpts(is_show=False,type_="inside",xaxis_index=[0, 1],range_start=90,range_end=100,),
        opts.DataZoomOpts(is_show=True,xaxis_index=[0, 5],type_="slider",pos_bottom="5%",range_start=85,range_end=100,)],
                    tooltip_opts=opts.TooltipOpts(trigger="axis",axis_pointer_type="cross",background_color="rgba(245, 245, 245, 0.8)",border_width=1,border_color="#ccc",
                                                  textstyle_opts=opts.TextStyleOpts(color="#000"),),)
)
bar.load_javascript()
bar.render_notebook()

In [86]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.layouts import column
import pandas as pd
sym="TCS"

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
df=fd.filter(pl.col('symbol')==sym).to_pandas()
inc = df.close > df.open
dec = df.open > df.close

p = figure(x_axis_type="datetime", tools=TOOLS, width=1000, height=400,title=f"{sym} Candlestick", background_fill_color="#efefef")
p.segment(df.epoch, df.high, df.epoch, df.low, color="black")

p.vbar(df.epoch[dec], pd.Timedelta('16h'), df.open[dec], df.close[dec], color="#eb3c40")
p.vbar(df.epoch[inc], pd.Timedelta('16h'), df.open[inc], df.close[inc], fill_color="white",line_color="#49a3a3", line_width=2)

t = figure(x_axis_type="datetime", width=1000, height=90)
t.varea(x=df.epoch,y1=0,y2=df.volume)
range_tool = RangeTool(x_range=p.x_range, start_gesture="pan")
t.add_tools(range_tool)
show(column(p,t))

In [20]:
import altair as alt
source = fd.filter(symbol=sym)
brush = alt.selection_interval(encodings=['x'])
nearest = alt.selection_point(nearest=True, on="pointerover",fields=["epoch"], empty=False)
rules = alt.Chart(source).mark_rule(color="gray").encode(x="epoch:T",).transform_filter(nearest)
open_close_color = alt.condition("datum.open <= datum.close",alt.value("#06982d"),alt.value("#ae1325"))
base = alt.Chart(source).encode(alt.X('epoch:T').scale(domain=brush).axis(format='%m/%d', labelAngle=-45).title('Date'),tooltip=['open','high','low','close'],
                                color=open_close_color).transform_filter(brush).add_params(nearest)
rule = base.mark_rule().encode(alt.Y('low:Q').title('Price').scale(zero=False),alt.Y2('high:Q'))
bar = base.mark_bar().encode(alt.Y('open:Q'),alt.Y2('close:Q'))
lower =  alt.Chart(source).mark_area(opacity=.6).encode(x = 'epoch:T',y = 'volume:Q').properties(height=75,width=1200).add_params(brush)

((rule + bar + rules).properties(width=1200,height=400) & lower)

alt.VConcatChart(...)

In [ ]:
import shutil
shutil.copyfile('C:\\Users\\smishra14\\OneDrive - Stryker\\python\\algo.ipynb', 'C:\\Users\\smishra14\\OneDrive\\Python\\algo.ipynb')

In [ ]:
options = Options()
#options.add_argument('--headless=new')

driver = webdriver.Chrome(options=options)
driver.get(a_c)

time.sleep(1)
login_with_clien_id_x_path='//*[@id="login_client_id"]'
elem = driver.find_element(By.XPATH, login_with_clien_id_x_path)
elem.click()
time.sleep(1)
client_id_input_x_path='//*[@id="fy_client_id"]'
elem2 = driver.find_element(By.XPATH, client_id_input_x_path)
elem2.send_keys(f['ci'])
elem2.send_keys(Keys.RETURN)
time.sleep(1)

t=pyotp.TOTP(f['totp']).now()

driver.find_element(By.XPATH, '//*[@id="first"]').send_keys(t[0])
driver.find_element(By.XPATH, '//*[@id="second"]').send_keys(t[1])
driver.find_element(By.XPATH, '//*[@id="third"]').send_keys(t[2])
driver.find_element(By.XPATH, '//*[@id="fourth"]').send_keys(t[3])
driver.find_element(By.XPATH, '//*[@id="fifth"]').send_keys(t[4])
driver.find_element(By.XPATH, '//*[@id="sixth"]').send_keys(t[5])

driver.find_element(By.XPATH, '//*[@id="confirmOtpSubmit"]').click() 
time.sleep(1)

driver.find_element(By.ID,"verifyPinForm").find_element(By.ID,"first").send_keys(f['pin'][0])
driver.find_element(By.ID,"verifyPinForm").find_element(By.ID,"second").send_keys(f['pin'][0])
driver.find_element(By.ID,"verifyPinForm").find_element(By.ID,"third").send_keys(f['pin'][0])
driver.find_element(By.ID,"verifyPinForm").find_element(By.ID,"fourth").send_keys(f['pin'][0])

driver.find_element(By.XPATH,'//*[@id="verifyPinSubmit"]').click()
time.sleep(1)
newurl = driver.current_url
print(newurl)
auth_code = newurl[newurl.index('auth_code=')+10:newurl.index('&state')]
print(auth_code)
driver.close()

In [5]:
session = fyersModel.SessionModel(client_id = f['client_id'],secret_key = f['secret_key'],redirect_uri = f['redirect_uri'],response_type = f['response_type'],grant_type="authorization_code")
session.set_token(auth_code)
response=session.generate_token()
access_token=response['access_token']
with open('access.txt','w') as k:
    k.write(access_token)

In [ ]:
df=df.sort('epoch').with_columns(pl.col('close').rolling_mean(3,by='epoch').alias('cl 3ma'))

In [ ]:
df.with_columns(pl.col('close')-pl.col('cl 3ma'))